In [ ]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = "2"

import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.backend as K
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import *


In [ ]:
num_aminoAcids = {0:'A', 1:'C', 2:'E', 3:'D', 4:'G', 5:'F', 6:'I', 7:'H', 8:'K', 9:'M', 10:'L',
            11:'N', 12:'Q', 13:'P', 14:'S', 15:'R', 16:'T', 17:'W', 18:'V', 19:'Y', 20:'X'}
num_ss = {0:'L',1:'B',2:'E',3:'G',4:'I',5:'H',6:'S',7:'T'}
aminoAcid_I = {j:i+1 for i,j in num_aminoAcids.items()}
aminoAcid_I['<pad>'] = 0
aminoAcid_I['<S>'] = len(aminoAcid_I)
aminoAcid_I['<EOS>'] = len(aminoAcid_I)
ss_I = {j:i+1 for i,j in num_ss.items()}
ss_I['<pad>'] = 0
ss_I['X'] = len(ss_I)
ss_I['<S>'] = len(ss_I)
ss_I['<EOS>'] = len(ss_I)

trainDataPath = 'Data/Secondary_Structure_Train_Dataset.npz'
testDataPath = 'Data/Secondary_Structure_Test_Dataset.npz'

tmp = np.load('Data/Secondary_Structure_Motif_Antimotif.npz')
motifs = tmp['motifs']
antiMotifs = tmp['antimotifs']
len(motifs),len(antiMotifs)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
def load_data(file_path):
    data = np.load(file_path)
    sequences = data['sequences']
    pssms = data['pssms']
    secondary_structure = data['secondaryStrucs']

    in1 = np.zeros((sequences.shape[0], 702), dtype=np.int32)
    in2 = np.zeros((sequences.shape[0], 702,22), dtype=np.float32)
    in3 = np.zeros((sequences.shape[0], 702), dtype=np.int32)
    out = np.zeros((sequences.shape[0], 702), dtype=np.int32)
    for i in tqdm(range(sequences.shape[0])):
        seq = '-'
        in1[i,0] = aminoAcid_I['<S>']
        in3[i,0] = 1
        out[i,0] = ss_I['<S>']
        for j in range(sequences.shape[1]):
            if np.sum(sequences[i,j,:]) == 0:
                in1[i,j+1] = aminoAcid_I['<EOS>']
                in3[i,j+1] = j+2
                out[i,j+1] = ss_I['<EOS>']
                break
            in3[i,j+1] = j+2
            t = num_aminoAcids[np.argmax(sequences[i,j,:])]
            seq += t
            in1[i,j+1] = aminoAcid_I[t]
            out[i,j+1] = ss_I[num_ss[np.argmax(secondary_structure[i,j,:])]]
            if np.sum(secondary_structure[i,j,:]) == 0:
                out[i,j+1] = ss_I['X']
            in2[i,j+1] = pssms[i,j]
    in4 = np.where(in1!=0, 1, 0)[:,:,None]
    return in1, in3, in2, in4, out


In [ ]:
X1, X2, X3, X4, Y = load_data(trainDataPath)
X1_val = X1[12000:]
X2_val = X2[12000:]
X3_val = X3[12000:]
X4_val = X4[12000:]
Y_val = tf.one_hot(Y[12000:], 9)[:,:,1:]

X1 = X1[:12000]
X2 = X2[:12000]
X3 = X3[:12000]
X4 = X4[:12000]
Y = tf.one_hot(Y[:12000], 9)[:,:,1:]


In [ ]:
def shape_list(x):
    tmp = list(K.int_shape(x))
    tmp[0] = -1
    return tmp

def dot_product_attention(q, k, v):
    depth = K.int_shape(q)[-1]
    dots = tf.matmul(q, k, transpose_b=True) /  tf.sqrt(float(depth))
    logsumexp = tf.math.reduce_logsumexp(dots, axis=-1, keepdims=True)
    dots = tf.exp(dots - logsumexp)
    attn = tf.matmul(dots, v)
    return attn

def prepare_dpa(x, n_heads, d_head):
    s_l = K.int_shape(x)[1]
    x = tf.reshape(x, (-1, s_l, n_heads, d_head))
    x = tf.transpose(x, (0, 2, 1, 3))
    x = tf.reshape(x, (-1, s_l, d_head))
    return x

def post_dpa(x, n_heads, d_head):
    s_l = K.int_shape(x)[1]
    x = tf.reshape(x, (-1, n_heads, s_l, d_head))
    x = tf.reshape(x, (-1, s_l, n_heads*d_head))
    return x

def enc_dec_attention(x, y, d_feat, n_heads):
    d_head = d_feat // n_heads
    q = tfk.layers.TimeDistributed(tfk.layers.Dense(d_feat))(y)
    q = prepare_dpa(q, n_heads, d_head)
    k = tfk.layers.TimeDistributed(tfk.layers.Dense(d_feat))(x)
    k = prepare_dpa(k, n_heads, d_head)
    v = tfk.layers.TimeDistributed(tfk.layers.Dense(d_feat))(x)
    v = prepare_dpa(v, n_heads, d_head)
    x = dot_product_attention(q, k, v)
    x = post_dpa(x, n_heads, d_head)
    return x

def casual_attention(x, d_feat, n_heads):
    d_head = d_feat // n_heads
    q = tfk.layers.TimeDistributed(tfk.layers.Dense(d_feat))(x)
    q = prepare_dpa(q, n_heads, d_head)
    k = tfk.layers.TimeDistributed(tfk.layers.Dense(d_feat))(x)
    k = prepare_dpa(k, n_heads, d_head)
    v = tfk.layers.TimeDistributed(tfk.layers.Dense(d_feat))(x)
    v = prepare_dpa(v, n_heads, d_head)
    x = dot_product_attention(q, k, v)
    x = post_dpa(x, n_heads, d_head)
    return x

def feed_forward(x, n_units, d_ff, d_rate):
    x = tfk.layers.LayerNormalization()(x)
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(d_ff, activation='relu'))(x)
    x = tfk.layers.Dropout(d_rate)(x)
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(n_units, activation='relu'))(x)
    x = tfk.layers.Dropout(d_rate)(x)
    return x

def decoder_block(x, z, n_units, d_ff, n_heads, d_rate):
    x = tfk.layers.LayerNormalization()(x)
    y = casual_attention(x, n_units, n_heads)
    y = tfk.layers.Dropout(d_rate)(y)
    x = tfk.layers.add([x,y])
    x = tfk.layers.LayerNormalization()(x)
    y = enc_dec_attention(z, x, n_units, n_heads)
    y = tfk.layers.Dropout(d_rate)(y)
    x = tfk.layers.add([x,y])
    y = feed_forward(x, n_units, d_ff, d_rate)
    y = tfk.layers.add([x,y])
    return y

def encoder_block(x, n_units, d_ff, n_heads, d_rate):
    x = tfk.layers.LayerNormalization()(x)
    y = casual_attention(x, n_units, n_heads)
    y = tfk.layers.Dropout(d_rate)(y)
    x = tfk.layers.add([x,y])
    y = feed_forward(x, n_units, d_ff, d_rate)
    y = tfk.layers.add([x,y])
    return y

def _getPosEncodingMat(length, dim):
    posEnc = np.array([[pos/np.power(10000, 2*(j//2)/dim) for j in range(dim)]
                        if pos!=0 else np.zeros(dim) for pos in range(length)], dtype=np.float32)
    posEnc[1:, 0::2] = np.sin(posEnc[1:, 0::2])
    posEnc[1:, 1::2] = np.cos(posEnc[1:, 1::2])
    return posEnc

In [ ]:
with strategy.scope():
    input1_ = tfk.layers.Input(shape=(702, ), name='sequence_input')
    input2_ = tfk.layers.Input(shape=(702, ), name='pids_input')
    input3_ = tfk.layers.Input(shape=(702, 22, ), name='pssm_input')
    input4_ = tfk.layers.Input(shape=(702, 1, ), name='mask_input')
    pidsEmbd = tfk.layers.Embedding(input_dim=702, output_dim=100, trainable=False,
                                  weights=[_getPosEncodingMat(702, 100)], name='pids_embds')(input2_)
    seqEmbd = tfk.layers.Embedding(input_dim=24, output_dim=100, name='seq_embds')(input1_)
    x = tfk.layers.Add(name='seq_embdAdd')([seqEmbd, pidsEmbd])
    x = tfk.layers.concatenate([x, input3_])
    x = tfk.layers.Conv1D(200, 11, strides=1, padding='same', activation='relu')(x)
    x = tfk.layers.Multiply()([x, input4_])
    for _ in range(3):
        x = encoder_block(x, 200, 512, 4, 0.2)
    output_ = tfk.layers.TimeDistributed( tfk.layers.Dense(256, activation='relu'))(x)
    output_ = tfk.layers.TimeDistributed( tfk.layers.Dense(8, activation='softmax'),name='output')(output_)
    
    model = tfk.models.Model([input1_, input2_, input3_, input4_], output_)
    model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
    model.summary()


In [ ]:
# model.fit([X1, X2, X3, X4], Y, verbose=1, batch_size=8, epochs=50)
# model.save_weights('Weights/BERT.h5')


In [ ]:
model.load_weights('Weights/BERT.h5')

In [ ]:
preds = model.predict([X1_val, X2_val, X3_val, X4_val], verbose=1, batch_size=8)
np.savez_compressed('bert-vals', val_tr=Y_val, val_pr=preds)

In [ ]:
def to_q3(x):
    y = []
    for i in x:
        if i in [0,6,7]:
            y += [1]
        elif i in [1,2]:
            y += [2]
        else:
            y += [3]
    return y

m = np.sum(Y_val, axis=-1)
y_t = np.argmax(Y_val[m==1],axis=-1)
y_p = np.argmax(preds[m==1],axis=-1)
print(classification_report(y_t,y_p))
print(accuracy_score(to_q3(y_t),to_q3(y_p)),accuracy_score(y_t,y_p),precision_score(y_t,y_p,average='weighted'),
      recall_score(y_t,y_p,average='weighted'), f1_score(y_t,y_p,average='weighted'))

In [ ]:
X1_te, X2_te, X3_te, X4_te, Y_te = load_data(testDataPath)
Y_te = tf.one_hot(Y_te, 9)[:,:,1:]


In [ ]:
preds = model.predict([X1_te, X2_te, X3_te, X4_te], verbose=1, batch_size=8)
np.savez_compressed('bert-tests', te_tr=Y_te, te_pr=preds)

In [ ]:
m = np.sum(Y_te, axis=-1)
y_t = np.argmax(Y_te[m==1],axis=-1)
y_p = np.argmax(preds[m==1],axis=-1)
print(classification_report(y_t,y_p))
print(accuracy_score(to_q3(y_t),to_q3(y_p)),accuracy_score(y_t,y_p),precision_score(y_t,y_p,average='weighted'),
      recall_score(y_t,y_p,average='weighted'), f1_score(y_t,y_p,average='weighted'))